In [ ]:
ANIMAL = "chimp"

In [ ]:
import pathlib
import keras
import tensorflow as tf
import plotly.graph_objects as go
import contextlib
import io

from petface.const import IMG_HEIGHT, IMG_WIDTH, ROOT_DIR
from petface.face_identification.model import get_reidentification_model

with contextlib.redirect_stdout(io.StringIO()):
    IMG_DIMENSIONS = (IMG_HEIGHT, IMG_WIDTH)

    root_dir = pathlib.Path(ROOT_DIR)
    test_dir = root_dir / "dataset" / "out" / "images" / "test" / ANIMAL

    test_ds = keras.utils.image_dataset_from_directory(
        directory=test_dir,
        image_size=IMG_DIMENSIONS,
        batch_size=32
    )

    eval_ds = test_ds.prefetch(tf.data.AUTOTUNE)  # type: ignore

In [ ]:
with contextlib.redirect_stdout(io.StringIO()):
    softmax_cross_entropy_model = get_reidentification_model(
        animal=ANIMAL, loss_type="cross_entropy", classification_type="softmax"
    )
    softmax_similarity_model = get_reidentification_model(
        animal=ANIMAL, loss_type="cross_entropy", classification_type="cosine_similarity"
    )
    arcface_cross_entropy_model = get_reidentification_model(
        animal=ANIMAL, loss_type="arcface", classification_type="softmax"
    )
    arcface_similarity_model = get_reidentification_model(
        animal=ANIMAL, loss_type="arcface", classification_type="cosine_similarity"
    )

    softmax_cross_entropy_model.load_from_checkpoint()
    softmax_similarity_model.load_from_checkpoint()
    arcface_cross_entropy_model.load_from_checkpoint()
    arcface_similarity_model.load_from_checkpoint()

In [ ]:
softmax_cross_entropy_model.display_training_history()

In [ ]:
softmax_similarity_model.display_training_history()

In [ ]:
arcface_cross_entropy_model.display_training_history()

In [ ]:
arcface_similarity_model.display_training_history()

In [ ]:
with contextlib.redirect_stdout(io.StringIO()):
    softmax_cross_entropy_eval_results = softmax_cross_entropy_model.evaluate(eval_ds)
    softmax_similarity_eval_results = softmax_similarity_model.evaluate(eval_ds)
    arcface_cross_entropy_eval_results = arcface_cross_entropy_model.evaluate(eval_ds)
    arcface_similarity_eval_results = arcface_similarity_model.evaluate(eval_ds)

In [ ]:
eval_table = go.Figure(
    data=[
        go.Table(
            header={"values": ["Loss", "Classifier", "Top 1 Accuracy", "Top 5 Accuracy", "Top 50 Accuracy"]},
            cells={
                "values": [
                    ["Cross Entropy", "Cross Entropy", "Arcface", "Arcface"],
                    ["Softmax", "Cosine Similarity", "Softmax", "Cosine Similarity"],
                    [
                        softmax_cross_entropy_eval_results["top_1_accuracy"],
                        softmax_similarity_eval_results["top_1_accuracy"],
                        arcface_cross_entropy_eval_results["top_1_accuracy"],
                        arcface_similarity_eval_results["top_1_accuracy"]
                    ],
                    [
                        softmax_cross_entropy_eval_results["top_5_accuracy"],
                        softmax_similarity_eval_results["top_5_accuracy"],
                        arcface_cross_entropy_eval_results["top_5_accuracy"],
                        arcface_similarity_eval_results["top_5_accuracy"]
                    ],
                    [
                        softmax_cross_entropy_eval_results["top_50_accuracy"],
                        softmax_similarity_eval_results["top_50_accuracy"],
                        arcface_cross_entropy_eval_results["top_50_accuracy"],
                        arcface_similarity_eval_results["top_50_accuracy"]
                    ],
                ]
            }
        )
    ]
)

eval_table.show()